[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Image Processing with Python

## Box Blur

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions



## Image Filtering

Image Filtering is the set of operations which alters image pixels values based on _spatial operations_.  
The operators are usually categorized by 2 main properties:

 - Linearity: $f \left( \alpha x + \beta y \right) = \alpha f \left( x \right) + \beta f \left( y \right)$.
 - Shift Invariance: $S \left( f \left( x \right) \right) = f \left( S \left( x \right) \right)$ where $S \left( \cdot \right)$ is a _Shift Operator_. 

Filters which are both _linear_ and _shift invariant_ (LSI) are applied using [Convolution](https://en.wikipedia.org/wiki/Convolution).

Given a realization of a filter, it can be also analyzed using concepts of _Signal Processing_.  
Namely its effect on the image _Spectrum_. Specifically if its operation is one of:
 - _Low Pass Filter_ (LPF): Removal of high frequency details.
 - _High Pass Filter_ (HPF): Removal of low frequency details.

This notebooks implements a simple LSI filter, the _Box Blur_.

* <font color='brown'>(**#**)</font> An LSI filter can be applied in Frequency Domain using the [Convolution Theorem](https://en.wikipedia.org/wiki/Convolution_theorem).
* <font color='brown'>(**#**)</font> Filters might be neither LPF nor HPF.

In [ ]:
# Parameters

# imgUrl = 'https://i.imgur.com/3BbIXdH.png' #<! A turtle climbing the Everest!
imgUrl = 'https://i.postimg.cc/63rN33GZ/3BbIXdH.png' #<! A turtle climbing the Everest!

boxRadius   = 5
lBoxRadius  = [1, 3, 5, 7, 9]

## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2) #<! Single channel image

In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title(f'Input Image');

## Box Blur

Box Blur is one of the most basic, yet useful, filters in Image Processing.  
It basically maps a rectangle rolling window to its mean value.

![](https://github.com/FixelAlgorithmsTeam/FixelCourses/raw/master/ImageProcessingMethods/Assets/BoxBlur.svg)

This section implements a _Box Blur_ in 3 different implementations:
 - Using [_Integral Image_](https://en.wikipedia.org/wiki/Summed-area_table) / [_Running Sum_](https://en.wikipedia.org/wiki/Running_total).
 - Using SciKit Image's [`skimage.util.view_as_windows()`](https://scikit-image.org/docs/stable/api/skimage.util.html#skimage.util.view_as_windows).

The reference is given by SciPy's built in function [`scipy.ndimage.uniform_filter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter.html).

* <font color='brown'>(**#**)</font> The blurring "quality" of the Box Blur is considered to be low (The roll off on the frequency domain).
* <font color='brown'>(**#**)</font> In some cases the output is the sum and not the mean value.
* <font color='brown'>(**#**)</font> The Box Blur is a [Separable Filter](https://en.wikipedia.org/wiki/Separable_filter).


### Implementation Using Running Sum / Integral Image

This section implements the Box Blur using either Integral Image or Running Sum.

#### Running Sum / Mean

Given a samples $\boldsymbol{x} = {\left[ 0.2, 0.7, 0.9, 0.5, 0.6 \right]}^{T}$.  
For a window of size 3, the first valid window is centered at the 2nd element: $\boldsymbol{w}_{2} = {\left[ 0.2, 0.7, 0.9 \right]}^{T}$.
The sum value is given by: ${w}_{2} = \boldsymbol{1}^{T} \boldsymbol{w}_{2}$.  
For the next window ${w}_{3} = {w}_{2} + {x}_{4} - {x}_{1}$. In general ${w}_{k} = {w}_{k - 1} + {x}_{k + r} - {w}_{k - 1 - r}$ where $r$ is the window radius.

#### Integral Image

The _Integral Image_ of $\boldsymbol{I}$ is given by:

$$ {T}_{m, n} = \sum_{i = 1, j = 1}^{m , n} {I}_{i, j} $$

To calculate the local sum one need to account the following:

![](https://i.postimg.cc/ZKq4WMLg/Box-filter-calculation-using-the-integral-image-the-shaded-area-indicates-the-filter-to.png)
<!-- ![](https://i.imgur.com/HFhNKoQ.png) -->

See [Steve Eddins - Fast Local Sums, Integral Images and Integral Box Filtering](https://blogs.mathworks.com/steve/2023/01/09/integral-image-box-filtering).


* <font color='red'>(**?**)</font> Think of possible issues with the Integral Image / Summed Area Table. Specifically consider large size image and the data format.  

<!-- Images of type `UInt8` / `UInt16` / `UInt32` might overflow for large images. Using `Float16` / `Float32` and even `Float64` might cause inaccuracies. -->

In [ ]:
# Box Blur by Integral Image / Running Sum

#===========================Fill This===========================#
# 1. Implement the Box Blur as a function using either Running Sum or Integral Image.
# 2. The function parameters are `mI`, `boxRadius`.
# 3. Pad the input properly, if needed, using `np.pad()` with `mode = 'edge'`.
# !! Use the separability property of the Box Blur.
# !! You may find .

def BoxBlur( mI: np.ndarray, boxRadius: int ) -> np.ndarray:
    """
    Applies a box blur to an input image using the integral image approach for efficient computation.

    Parameters
    ----------
    mI : np.ndarray
        Input 2D grayscale image as a numpy array. The image can have any numeric `dtype`,
        including `np.uint8`, `np.int32`, `np.float32`, etc.
    boxRadius : int
        Radius of the box kernel. The box kernel size will be `2 * boxRadius + 1`.
        For example, if `boxRadius` is 1, the kernel size will be 3x3.

    Returns
    -------
    mO : np.ndarray
        Blurred output image of the same shape as the input image, with each pixel representing
        the average intensity within a box of size `(2 * boxRadius + 1) x (2 * boxRadius + 1)` centered
        around that pixel. The output array has `dtype` of `np.float64`.

    Notes
    -----
    This function leverages the concept of an integral image (summed-area table) to efficiently calculate
    the sum of pixel values within rectangular regions. The integral image allows computation of box
    sums in constant time, making this method much faster than direct convolution with a box kernel.

    The function first pads the input image symmetrically to handle edge pixels, then computes
    the integral image, and finally extracts box-sum regions by subtracting appropriate corners
    of the integral image for each box.

    Raises
    ------
    TypeError
        If the input `mI` does not have an integer or floating-point `dtype`.

    Examples
    --------
    >>> import numpy as np
    >>> mI = np.array([
    ...     [50, 80, 100, 120, 150],
    ...     [60, 90, 110, 130, 160],
    ...     [70, 100, 120, 140, 170],
    ...     [80, 110, 130, 150, 180],
    ...     [90, 120, 140, 160, 190]
    ... ], dtype=np.uint8)
    >>> boxRadius = 1  # 3x3 kernel
    >>> mB = BoxBlur(mI, boxRadius)
    >>> print(mB) #<! Blurred image
    """

    eltType = mI.dtype

    # See https://numpy.org/doc/stable/reference/arrays.scalars.html
    if np.issubdtype(eltType, np.unsignedinteger):
        outType = np.uint64
    elif np.issubdtype(eltType, np.signedinteger):
        outType = np.int64
    elif np.issubdtype(eltType, np.floating):
        outType = np.float64
    else:
        raise TypeError(f'The type of `mI` must be a sub dtype of `np.unsignedinteger`, `np.signedinteger` or `np.floating`')
    
    boxLen = 2 * boxRadius + 1

    mP = np.pad(mI, (boxRadius, boxRadius), mode = 'edge')
    mS = np.cumsum(np.cumsum(mP, axis = 0, dtype = outType), axis = 1, dtype = outType)
    mS = np.pad(mS, ((1, 0), (1, 0)), mode = 'constant', constant_values = 0)

    # Define the shifted slices for each corner of the kernel window
    mTL = mS[:-boxLen, :-boxLen] #<! Top left
    mTR = mS[:-boxLen, boxLen:] #<! Top right
    mBL = mS[boxLen:, :-boxLen] #<! Bottom left
    mBR = mS[boxLen:, boxLen:] #<! Bottom right

    mO = (mBR - mTR - mBL + mTL) / (boxLen * boxLen)

    return mO

#===============================================================#

In [ ]:
# Box Blur by SciKit Image `view_as_windows()`

#===========================Fill This===========================#
# 1. Implement the Box Blur as a function using `skimage.util.view_as_windows()`.
# 2. The function parameters are `mI`, `boxRadius`.
# 3. Pad the input properly, if needed, using `np.pad()` with `mode = 'edge'`.
# !! Use the separability property of the Box Blur.

def BoxBlurView( mI: np.ndarray, boxRadius: int ) -> np.ndarray:    
    """
    Computes the box blur (average blur) of an input image using a running window approach.

    This function applies a box blur filter to the image by averaging pixel values within a square window
    of size `(2 * boxRadius + 1) x (2 * boxRadius + 1)` around each pixel. It uses `view_as_windows` from
    `skimage.util` to create sliding windows, making the computation efficient without explicit loops.

    Parameters
    ----------
    mI : np.ndarray
        The input grayscale image represented as a 2D numpy array.
    boxRadius : int
        The radius of the box (blur) filter. The resulting window size will be `(2 * boxRadius + 1) x (2 * boxRadius + 1)`.

    Returns
    -------
    np.ndarray
        A 2D numpy array of the same shape as `mI`, containing the blurred image.

    Notes
    -----
    - The function pads the input image symmetrically using edge padding to handle borders, so the output
      has the same size as the input.
    - The function uses a mean calculation over the window dimensions, creating a box blur effect.

    Example
    -------
    >>> import numpy as np
    >>> from skimage import data
    >>> mI = data.camera()
    >>> mB = BoxBlurView(mI, boxRadius = 3)
    """

    mP = np.pad(mI, (boxRadius, boxRadius), mode = 'edge')
    tW = ski.util.view_as_windows(mP, (2 * boxRadius + 1, 2 * boxRadius + 1))
    mO = np.mean(tW, axis = (2, 3))

    return mO

#===============================================================#

* <font color='brown'>(**#**)</font> The function `view_as_windows()` is mainly for Non Linear and / or Shift Variant operators which can not be implemented using _Convolution_.

* <font color='green'>(**@**)</font> Create a running sum version of the box blur using `Numba` for acceleration.

In [ ]:
mORef = sp.ndimage.uniform_filter(mI, 2 * boxRadius + 1, mode = 'nearest') #<! Reference
mOi = BoxBlur(mI, boxRadius) #<! Integral images
mOv = BoxBlurView(mI, boxRadius)

In [ ]:
print(np.max(np.abs(mORef - mOi)))
print(np.max(np.abs(mORef - mOv)))